In [1]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers, models
import pennylane as qml
import numpy as np
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds

2024-05-07 15:06:33.267411: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-07 15:06:33.268710: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-07 15:06:33.294725: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-07 15:06:33.295421: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 15:06:33.778477: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
# Check hardware resources
print("TensorFlow Version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow Version: 2.12.0
GPU Available: []


2024-05-07 15:06:35.113777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-07 15:06:35.114118: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
# Global parameters
BATCH_SIZE = 32
EPOCHS = 10

In [4]:
# Load and preprocess datasets
def load_plantdoc():
    """Load and preprocess the PlantDoc dataset."""
    classes = sorted([
        'Apple Scab Leaf', 'Apple leaf', 'Apple rust leaf', 'Bell_pepper leaf', 'Bell_pepper leaf spot',
        'Blueberry leaf', 'Cherry leaf', 'Corn Gray leaf spot', 'Corn leaf blight', 'Corn rust leaf',
        'Peach leaf', 'Potato leaf early blight', 'Potato leaf late blight', 'Raspberry leaf',
        'Soyabean leaf', 'Squash Powdery mildew leaf', 'Strawberry leaf', 'Tomato Early blight leaf',
        'Tomato Septoria leaf spot', 'Tomato leaf', 'Tomato leaf bacterial spot', 'Tomato leaf late blight',
        'Tomato leaf mosaic virus', 'Tomato leaf yellow virus', 'Tomato mold leaf',
        'Tomato two spotted spider mites leaf', 'grape leaf', 'grape leaf black rot'
    ])
    num_classes = len(classes)
    class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
    
    def load_images_from_folder(folder):
        images = []
        labels = []
        for class_name in os.listdir(folder):
            class_folder = os.path.join(folder, class_name)
            if os.path.isdir(class_folder):
                for image_name in os.listdir(class_folder):
                    image_path = os.path.join(class_folder, image_name)
                    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(128, 128))
                    image = tf.keras.preprocessing.image.img_to_array(image)
                    images.append(image)
                    labels.append(class_to_idx[class_name])
        return np.array(images), np.array(labels)
    
    train_folder = 'data/train'
    test_folder = 'data/test'
    
    X_train, y_train = load_images_from_folder(train_folder)
    X_test, y_test = load_images_from_folder(test_folder)
    
    X_train = X_train / 255.0
    X_test = X_test / 255.0
    
    return (X_train, y_train), (X_test, y_test), num_classes

def load_plantvillage():
    """Load the PlantVillage dataset."""
    dataset, info = tfds.load('plant_village', with_info=True, as_supervised=True)
    train_dataset, test_dataset = dataset['train'], dataset['test']

    def normalize_img(image, label):
        """Normalizes images to [0, 1]."""
        return tf.cast(image, tf.float32) / 255.0, label

    train_dataset = train_dataset.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
    test_dataset = test_dataset.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

    return train_dataset, test_dataset, info.features['label'].num_classes

def load_cifar10():
    """Load CIFAR-10 dataset."""
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
    X_train = X_train / 255.0
    X_test = X_test / 255.0
    return (X_train, y_train), (X_test, y_test), 10

def load_cifar100():
    """Load CIFAR-100 dataset."""
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar100.load_data()
    X_train = X_train / 255.0
    X_test = X_test / 255.0
    return (X_train, y_train), (X_test, y_test), 100

In [5]:
# Dataset loading functions
datasets_dict = {
    'plantdoc': load_plantdoc#,
    #'plantvillage': load_plantvillage#,
    #'cifar10': load_cifar10,
    #'cifar100': load_cifar100
}

In [6]:
# Model definitions
def create_simple_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def create_resnet50(input_shape, num_classes):
    base_model = tf.keras.applications.ResNet50(include_top=False, weights=None, input_shape=input_shape)
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def create_vgg16(input_shape, num_classes):
    base_model = tf.keras.applications.VGG16(include_top=False, weights=None, input_shape=input_shape)
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def create_densenet121(input_shape, num_classes):
    base_model = tf.keras.applications.DenseNet121(include_top=False, weights=None, input_shape=input_shape)
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def create_qcnn(input_shape, num_classes):
    """Quantum CNN using PennyLane."""
    dev = qml.device('default.qubit', wires=4)

    @qml.qnode(dev)
    def circuit(inputs, weights):
        qml.templates.AngleEmbedding(inputs, wires=range(4))
        qml.templates.StronglyEntanglingLayers(weights, wires=range(4))
        return [qml.expval(qml.PauliZ(i)) for i in range(4)]

    n_qubits = 4
    q_depth = 2
    weight_shapes = {"weights": (q_depth, n_qubits, 3)}

    qcnn_model = qml.qnn.KerasLayer(circuit, weight_shapes, output_dim=num_classes)
    cnn_model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(n_qubits),
        qcnn_model,
        layers.Dense(num_classes, activation='softmax')
    ])
    return cnn_model

In [7]:
# Dictionary of models
models_dict = {
    'simple_cnn': create_simple_cnn,
    'resnet50': create_resnet50,
    'vgg16': create_vgg16,
    'densenet121': create_densenet121,
    'qcnn': create_qcnn
}

In [8]:
# Objective function for hyperparameter optimization using Optuna
def objective(trial):
    dataset_name = trial.suggest_categorical('dataset', list(datasets_dict.keys()))
    model_name = trial.suggest_categorical('model', list(models_dict.keys()))
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])

    (X_train, y_train), (X_test, y_test), num_classes = datasets_dict[dataset_name]()
    input_shape = X_train.shape[1:]

    # Instantiate the selected model
    model = models_dict[model_name](input_shape, num_classes)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS, verbose=0)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [9]:
# Run optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-05-07 15:06:35,175] A new study created in memory with name: no-name-7ac3b24c-a4b0-41c3-8462-92ca751ac9ef
/tmp/ipykernel_11599/2744277618.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)


In [ ]:
# Print best trial details
print("Best Trial:")
print(" Value:", study.best_trial.value)
print(" Params:", study.best_trial.params)

Best Trial:


ValueError: No trials are completed yet.

In [ ]:
# Training plot
fig, ax = plt.subplots()
optuna.visualization.plot_optimization_history(study).show()